# Final Assignment 

## Instuctions
* An extra 20 points for those who implement ALL the functions themselves without using tensorflows tf.keras.preprocessing and TextVectorization.
* Use pyarrow to save the embedding map.
* Find most similar function should be efficient. 20 sec to wait for a result is too much. You should aim to < 2 sec (use "timeit" magic to verify)
* Output cells should not be too big. DO NOT DUMP A LOT OF DATA IN THE OUTPUT. NOTEOOKS THAT WON'T FOLLOW THIS INSTRUCTION WILL NOT BE CHECKED AND THEIR GRADE WILL BE SET TO ZERO.

### Read dataset 

In [1]:
import zipfile
import pandas as pd

zip_path = 'data/review_230k.zip'

def extract_parquet_from_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(f"Files in zip: {file_list}") 

        parquet_file_name = [f for f in file_list if f.endswith('.parquet')][0]

        with zip_ref.open(parquet_file_name) as file:
            df = pd.read_parquet(file)
    return df

data = extract_parquet_from_zip(zip_path)



Files in zip: ['review_230k.parquet']


In [2]:
def create_dataset(data: pd.DataFrame):
    """ Create the dataset in your preferrable format """
    return data

In [3]:
dataset = create_dataset(data)

#
print(dataset.head())

                                           title  \
0        Truly is "Jewel of the Upper Wets Side"   
1                        My home away from home!   
2                                     Great Stay   
3                             Modern Convenience   
4  Its the best of the Andaz Brand in the US....   

                                                text  
0  Stayed in a king suite for 11 nights and yes i...  
1  On every visit to NYC, the Hotel Beacon is the...  
2  This is a great property in Midtown. We two di...  
3  The Andaz is a nice hotel in a central locatio...  
4  I have stayed at each of the US Andaz properti...  


### Clean and standardize the data

In [4]:

import string

def prepare_dataset(dataset: pd.DataFrame):
  
    dataset['combined_text'] = dataset['title'] + ' ' + dataset['text']
    def clean_text(text):
       
        text = text.lower()
        # Remove punctuation and numbers
        text = text.translate(str.maketrans('', '', string.punctuation + string.digits))
        # split text into sentences
        sentences = text.split('.')
        
        # split sentences to words
        sentence_word_arrays = [sentence.split() for sentence in sentences if sentence]
        
        return sentence_word_arrays

    # cleaning all text
    dataset['tokenized_sentences'] = dataset['combined_text'].apply(clean_text)

    return dataset['tokenized_sentences'].tolist()

binary_sentences = prepare_dataset(dataset)
for sentence_array in binary_sentences[:5]:
    print(sentence_array)


[['truly', 'is', 'jewel', 'of', 'the', 'upper', 'wets', 'side', 'stayed', 'in', 'a', 'king', 'suite', 'for', 'nights', 'and', 'yes', 'it', 'cots', 'us', 'a', 'bit', 'but', 'we', 'were', 'happy', 'with', 'the', 'standard', 'of', 'room', 'the', 'location', 'and', 'the', 'friendliness', 'of', 'the', 'staff', 'our', 'room', 'was', 'on', 'the', 'th', 'floor', 'overlooking', 'broadway', 'and', 'the', 'madhouse', 'of', 'the', 'fairway', 'market', 'room', 'was', 'quite', 'with', 'no', 'noise', 'evident', 'from', 'the', 'hallway', 'or', 'adjoining', 'rooms', 'it', 'was', 'great', 'to', 'be', 'able', 'to', 'open', 'windows', 'when', 'we', 'craved', 'fresh', 'rather', 'than', 'heated', 'air', 'the', 'beds', 'including', 'the', 'fold', 'out', 'sofa', 'bed', 'were', 'comfortable', 'and', 'the', 'rooms', 'were', 'cleaned', 'well', 'wifi', 'access', 'worked', 'like', 'a', 'dream', 'with', 'only', 'one', 'connectivity', 'issue', 'on', 'our', 'first', 'night', 'and', 'this', 'was', 'promptly', 'respond

### Create A vocabulary 

In [ ]:
def create_vocabulary(dataset) -> dict[str, int]:
 
    vocabulary = {} 
    for sentences in dataset:
        for sentence in sentences:
            for word in sentence:
                if word in vocabulary:
                    vocabulary[word] += 1
                else:
                    vocabulary[word] = 1


    sorted_vocabulary = dict(sorted(vocabulary.items(), key=lambda item: item[1], reverse=True))

    return sorted_vocabulary  
vocabulary = create_vocabulary(binary_sentences)

# Вывод первых 10 самых частых слов
for word, frequency in list(vocabulary.items())[:40]:
    print(f"{word}: {frequency}")


### Word encoding 

In [ ]:
def encode_dataset(dataset, vocabulary: dict[str, int]):
    """ Encode each word in the dataset, based on your vocabulary """
    pass

encoded_dataset = encode_dataset(dataset, vocabulary)

Example using tensoflow 

In [ ]:

from tensorflow.keras import layers

# you can also use "adapt" 
vocabulary = create_vocabulary(dataset) 

vectorize_layer = layers.TextVectorization(
    max_tokens=...,
    output_mode='int',
    output_sequence_length=...,
    vocabulary=vocabulary
)

# adds padding and [UNK] token
vocabulary = vectorize_layer.get_vocabulary()

encoded_text_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
encoded_dataset = encoded_text_ds.as_numpy_iterator()


### Generate Positive and Negative Pairs 

In [ ]:
def generate_training_data(encoded_dataset, number_of_neg_samples: int):
    """ 
    Generate positive and negative pairs 
    param: encoded_dataset:  the encoded dataset
    param: number_of_neg_samples: Number of negative samples per positive pair 
    """
    pass

Example using tensoflow 

In [ ]:
# understand what is the functionalify of this, before you use it.
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(len(vocabulary))


# Generate positive skip-gram pairs for a sequence
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      sequence,
      vocabulary_size=VOCABULARY_SIZE,
      sampling_table=sampling_table,
      window_size=...,
      negative_samples=0)


# Iterate over each positive skip-gram pair to produce training examples
# with a positive context word and negative samples.
for target_word, context_word in positive_skip_grams:
    
    context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)
    negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
      true_classes=context_class,
      num_true=1,
      num_sampled=NUMBER_OF_NEGATIVE_SAMPLES,
      unique=True,
      range_max=VOCABULARY_SIZE,
      seed=123,
      name="negative_sampling")

    # Build context and label vectors (for one target word)
    context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
    label = tf.constant([1] + [0] * NUMBER_OF_NEGATIVE_SAMPLES, dtype="int64")

    # Append each element from the training example to global lists.
    targets.append(target_word)
    contexts.append(context)
    labels.append(label)



### Define the model 

In [ ]:
 # define you model here 

### Training

Example, there are other ways to train like using gradient tape

In [ ]:
model.compile(optimizer=..., loss=..., metrics=[...])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="model_logs")

# checkpoint a model. here we save the best model relative to validation loss
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath="word2vec_model_w5_ns15_ckpt.h5", monitor=..., save_best_only=True)

# restore_best_weights - Whether to restore model weights from
# the epoch with the best value of the monitored quantity.
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='accuracy')

history = model.fit(dataset, epochs=..., callbacks=[tensorboard_callback, checkpoint_cb, early_stopping_cb])

model.save(f"word2vec_model_w{WINDOW_SIZE}_ns{NEGATIVE_SAMPLES}.h5", include_optimizer=True)

### Save embeddings 

In [ ]:

def save_embeddings(model, vocabulary: List[str], path: str):
    """ 
    Save the embeddings 
    param: model: the trained tf model 
    param: vocabulary: list of tokens 
    param: path: the path to save the embeddings map 

    hint: take the weights using model.get_layer(...).get_weights()
    """
    pass

### Find Most similar

Note that this must be efficient 

In [ ]:
def find_most_similar(word: str, k: int = 10) -> List[Tuple[str, float]]:
    """ 
    Find most similar tokens to word 
    param: word: the word to find most similar words to 
    k: number of most similar words 
    """
    pass

    

### Dimentionality Reduction 

visualize some clusters (pick some subset of words to show the labels for)